# paiza スキルチェックの成績を取得する

- 研究室の paiza 課題のために，評価結果を一気に Excel 形式にまとめるためのツール
- paiza スキルチェックの[評価結果一覧画面](https://paiza.jp/student/mypage/results) の内容を集計する

- メモ
  - selenium 4 は Google Colab 上でバージョン不整合が起きる（？）ため selenium 3 を指定

In [ ]:
!pip install selenium~=3.14
!apt-get update
!apt install chromium-chromedriver

In [ ]:
def get_paiza_results():
  '''get_paiza_results
    paiza の評価結果一覧ページを取得する。
    戻り値は HTML ソース文字列
  '''

  import time

  from getpass import getpass
  from selenium import webdriver
  from selenium.webdriver.common.by import By
  #from selenium.webdriver.support.ui import WebDriverWait
  #from selenium.webdriver.support import expected_conditions as EC

  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')

  driver = webdriver.Chrome('chromedriver', options=options)

  driver.implicitly_wait(10)

  url_result = 'https://paiza.jp/student/mypage/results'
  url_signin = 'https://paiza.jp/sign_in'

  driver.get(url_result)
  print('評価結果一覧ページを開いています...')
  time.sleep(2)

  if driver.current_url == url_signin:
    # 電子メールアドレス
    email = input('paiza の電子メールアドレスを入力してください').strip()
    elem = driver.find_element(by=By.XPATH, value="//*[@type='email']")
    elem.send_keys(email)
    print(elem.get_attribute('value'))

    # パスワード
    password = getpass('paiza のパスワードを入力してください').strip()
    elem = driver.find_element(by=By.XPATH, value="//*[@type='password']")
    elem.send_keys(password)

    # ログインボタンを押す
    elem = driver.find_element(by=By.CLASS_NAME, value='a-button-primary-large')
    print(elem.is_enabled())
    elem.click()
  
    print('ログイン中です...')
    time.sleep(2)

  print(driver.current_url)
  assert driver.current_url == url_result
  print('ログインしました')

  return driver.page_source

In [ ]:
def parse_paiza_results(doc):
  '''parse_paiza_results
    paiza の評価結果一覧ページ（HTML）を解析し，DataFrame に変換する。
  '''

  import pandas as pd
  from bs4 import BeautifulSoup

  BASIC_BOX_CLASS = 'd-mypage-my-results__basicBox'
  TITLE_CLASS = 'd-mypage-my-results__box-top__title'
  BOX_TOP_CLASS = 'd-mypage-my-results__box-top'
  BOX_MIDDLE_CLASS = 'd-mypage-my-results__box-middle'
  BOX_BOTTOM_CLASS = 'd-mypage-my-results__box-bottom'

  soup = BeautifulSoup(doc)

  df_all = None

  for result in soup.select(f'div.{BASIC_BOX_CLASS}'):
    d = {}
    d['title'] = result.select_one(f'.{TITLE_CLASS}').text.strip()
    d['problem_id'] = result.select_one(f'.{TITLE_CLASS} a').attrs['href']
    d['submitted'] = result.select(f'.{BOX_TOP_CLASS} > span')[1].text.strip()

    temp = result.select(f'.{BOX_MIDDLE_CLASS} span')
    d['lang'] = temp[2].text.strip()
    d['time'] = temp[4].text.strip()
    d['score'] = temp[8].text.strip()

    temp = result.select(f'.{BOX_BOTTOM_CLASS} > span')
    d['difficulty'] = temp[2].text.strip()
    d['examinees'] = temp[4].text.strip()
    d['correct_answer_rate'] = temp[6].text.strip()
    d['average_time'] = temp[8].text.strip()
    d['average_score'] = temp[10].text.strip()

    if df_all is None:
      df_all = pd.Series(d).to_frame().T
    else:
      df_all = df_all.append(d, ignore_index=True)
  
  return df_all

In [ ]:
import pandas as pd

In [ ]:
doc = get_paiza_results()
df_orig = parse_paiza_results(doc)

In [ ]:
df = df_orig.copy()

# Check
n = len(df)
idx = df.time.str.match('[0-9]+分[0-9]+秒')
if sum(idx) != n: print(df.time[idx])
idx = df.score.str.match('[0-9]+点')
if sum(idx) != n: print(df.score[idx])

# Transform
df['problem_code'] = df.title.str.replace('.*([SABCD][0-9]+)(?=:).*', lambda m: m.group(1), regex=True)
df['title'] = df.title.str.replace('(.*)([SABCD][0-9]+:)(.*)', lambda m: m.group(1) + m.group(3), regex=True)
df['problem_id'] = df.problem_id.str.replace('.*/challenges/([0-9]+)/.*', lambda m: m.group(1), regex=True)
df['submitted'] = df.submitted.str.replace('提出日：(.*)', lambda m: m.group(1), regex=True)
df['time'] = df.time.str.replace('([0-9]+)分([0-9]+)秒', lambda m: str(int(m.group(1)) * 60 + int(m.group(2))), regex=True).astype(int)
df['score'] = df.score.str.replace('([0-9]+)点', lambda m: m.group(1), regex=True).astype(int)

# 研究室フォーマットへ
# ToDo: コードのサイズ（バイト）を取得する
df['Name'] = None
df['Bytes'] = None
df['Comment'] = None
df['submitted'] = pd.to_datetime(df['submitted']).dt.strftime('%Y/%m/%d %H:%M:%S')
df['time'] = df.time.clip(upper=24*60*60-1)
df['time'] = (pd.to_datetime(0) + pd.to_timedelta(df.time, unit='sec')).dt.strftime('%H:%M:%S')

df = df[['submitted', 'Name', 'problem_code', 'title', 'lang', 'time', 'Bytes', 'score', 'Comment']]
df.columns = 'Date Name Problem Title Lang Time Bytes Score Comment'.split()
#df = df.sort_values('Problem', key=lambda col: col.str.translate(str.maketrans({'D':'1', 'C':'2', 'B':'3', 'A':'4', 'S':'5'})))
df

In [ ]:
path = 'paiza.xlsx'
df.to_excel(path, index=False)
from google.colab import files
files.download(path)